**Recreating the plasmid figure**


The plasmid figure was done by running a multifasta file containing all plasmids in a database, and then recording how many of those plasmids were found by that method.

1. First, run your multifasta file of plasmids in skandiver, genomad, and mefinder. In the paper, the following database was used: https://datadryad.org/stash/dataset/doi:10.15146/R33X2J

The following commands were used to run skandiver, mefinder, and genomad on a multifasta file or directory containing all plasmids from that database:


```
#skandiver
./skandiver.sh all_plasmids/ all_plasmid_10k_results 10000 gtdb_skani_database_ani

#mefinder
mefinder find --contig all_plasmids/all_plasmids.fasta mefinder_all_plasmids_output

#genomad
genomad end-to-end --cleanup --splits 8 all_plasmids/all_plasmids_with_names.fasta genomad_all_plasmids_results genomad_db
```
2. Next, we will extract the names of all plasmids that were found by each method. We use the following command:

```
python3 /usr1/shared/find_gene.py plasmid_names.txt output_file.txt/.csv found_plasmids.txt
```
The first argument is a .txt file of the names of all plasmids, each on a new line. The second argument is the output file of the method you used, and the third is the name you want of your output file.

3. Now that all data has been collected from each method, we can graph the results. You will need a .csv file with the plasmid name and its length. Use the following code block / function below to plot the data:


In [ ]:
# if you don't have the data for one of the methods, just say None in its place
plot_hist(<plasmid_lengths>, <skandiver>, <genomad>, <mefinder>, <bucket_num(number of buckets you want in the histogram>)

In [ ]:
#@title Graph Plotting Function
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.patches as patches

def make_percentages(og_bucket_count, new_bucket):
    percentages = []
    for i in range(len(og_bucket_count)):
        per = (new_bucket[i]/og_bucket_count[i])*100
        #print("per", per)
        percentages.append(per)
    return percentages

def obtain_bucket_counts(plasmid_lengths_df, bin_edges, found_plasmids, num_buckets):
    found_bucket_counts = {i: 0 for i in range(num_buckets)}
    for length in plasmid_lengths_df[plasmid_lengths_df["Gene Name"].isin(found_plasmids)]["Length"]:
        bucket_index = find_index(length, bin_edges)
        #print(length)
        found_bucket_counts[bucket_index] += 1
    return found_bucket_counts

def find_index(length, bin_edges):
    for i in range(0, len(bin_edges)-1):
        if bin_edges[i] <= length and length <= bin_edges[i+1]:
            #print(f'{bin_edges[i]} <={length} <= {bin_edges[i+1]}', i)
            return i
    return len(bin_edges)-2

def plot_hist(plasmid_lengths, skandiver, genomad, mefinder, bucket_num):
    # Sample data for four different sets of bars
    skani_color =  (254/255, 17/255, 17/255)
    genomad_color =  (1/255, 207/255, 13/255)
    mefinder_color =  (1/255, 40/255, 218/255)

    bucket_arr = []
    for i in range(1, bucket_num+1):
        bucket_arr.append(i)
    x = np.array(bucket_arr)

    plasmid_lengths_df = pd.read_csv(plasmid_lengths)

    min_length = min(plasmid_lengths_df["Length"])
    max_length = max(plasmid_lengths_df["Length"])
    # Specify the number of buckets and bucket size
    num_buckets = bucket_num
    bin_edges = np.logspace(np.log10(min_length), np.log10(max_length), num=num_buckets+1)
    bucket_size = max(plasmid_lengths_df["Length"]) // num_buckets

    bucket_counts_original = {i: 0 for i in range(num_buckets)}
    for length in plasmid_lengths_df["Length"]:
        bucket_index = find_index(length, bin_edges)
        bucket_counts_original[bucket_index] += 1

    # Width of each bar
    width = 0.2

    genomad_color =  (1/255, 207/255, 13/255)

    if skandiver:
        with open(skandiver, "r") as file:  # Replace with the path to your found plasmids file
            found_skani_plasmids = [line.strip() for line in file]
        skandiver_count = obtain_bucket_counts(plasmid_lengths_df, bin_edges, found_skani_plasmids, bucket_num)
        skandiver_plot = make_percentages(bucket_counts_original, skandiver_count)
        y3 = np.array(skandiver_plot)
        plt.bar(x + 3 * width, y3, width=width, color='red', label='')

    if mefinder:
            with open(mefinder, "r") as file:  # Replace with the path to your found plasmids file
                found_mefinder_plasmids = [line.strip() for line in file]
            mefinder_count = obtain_bucket_counts(plasmid_lengths_df, bin_edges, found_mefinder_plasmids, bucket_num)
            mefinder_plot = make_percentages(bucket_counts_original, mefinder_count)
            y2 = np.array(mefinder_plot)
            # Plotting the second set of bars with a specific color
            plt.bar(x + width, y2, width=width, color='blue')

    if genomad:
            with open(genomad, "r") as file:  # Replace with the path to your found plasmids file
                found_genomad_plasmids = [line.strip() for line in file]
            genomad_count = obtain_bucket_counts(plasmid_lengths_df, bin_edges, found_genomad_plasmids, bucket_num)
            genomad_plot = make_percentages(bucket_counts_original, genomad_count)
            y4 = np.array(genomad_plot)
            # Plotting the fourth set of bars with a specific color
            plt.bar(x + 2*width, y4, width=width, color=(1/255, 207/255, 13/255))

    #print("mefinder", mefinder_plot)
    #print("genomad", genomad_plot)
    #print("skandiver", skandiver_plot)
    bucket_labels = []
    for i in range(bucket_num):
        bucket_labels.append(f"{bin_edges[i]}-{bin_edges[i+1]}")
    print("bin edges ", bucket_labels)
    # Labeling the x-axis
    plt.xlabel('Bin edges')

    # Labeling the y-axis
    plt.ylabel('Percent Plasmids Found')

    # Adding a title
    plt.title('Percent of Plasmids Found For each Method')

    plt.legend(handles=[patches.Patch(color=skani_color, label='skandiver'),
                    patches.Patch(color=mefinder_color, label='mefinder'),
                    patches.Patch(color=genomad_color, label='genomad')],
            bbox_to_anchor=(1.05, 1), loc='upper left')


    # Display the plot
    plt.show()